## Tutorial smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

In [ ]:
!pip install smolagents #transformers accelerate

In [55]:
# Para poder acceder al modelo de Hugging Face: google/gemma-2b-it
hf_key = ''
!huggingface-cli login --token $hf_key

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `jaz-read` has been saved to C:\Users\ensnn\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\ensnn\.cache\huggingface\token
Login successful.
The current active token is: `jaz-read`


In [56]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

 ### 1. Usando HF Inference API

In [77]:
from smolagents import CodeAgent, HfApiModel

model = HfApiModel(model_id=model_id, token=hf_key) # You can choose to not pass any model_id to HfApiModel to use a default free model
# you can also specify a particular provider e.g. provider="together" or provider="sambanova"
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      fib_sequence = [0, 1]                                                                                        
      while len(fib_sequence) < n:                                                                                 
          fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])                                                 
      return fib_sequence[-1]                                                                                      
                                                                                                                   
  result = fibonacci(119) # We need 119 because list indices start at 0                                            
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
2046711111473984623691759

Out: None

[Step 0: Duration 0.17 seconds| Input tokens: 2,127 | Output tokens: 139]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(2046711111473984623691759)                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 1: Duration 0.19 seconds| Input tokens: 4,544 | Output tokens: 190]

2046711111473984623691759

El resultado no es correcto.

En el siguiente ejemplo, se define la función de fibonacci como una tool y entonces ya da el resultado correcto:

In [84]:
from smolagents import tool

@tool
def fibonacci(n: int) -> int:
    """
    Calcula el enésimo número de Fibonacci.

    Args:
        n: Posición del número de fibonacci a calcular.
    """

    a, b = 0, 1
    for _ in range(n - 1):
        a, b = b, a + b
    return a

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 118th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = fibonacci(n=118)                                                                                        
  print(result)                                                                                                    
                                                                                                                   
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out - Final answer: 1264937032042997393488322

[Step 0: Duration 9.99 seconds| Input tokens: 2,171 | Output tokens: 128]

Response: 1264937032042997393488322


Otra forma de definir la tool pero usando una class

In [78]:
from smolagents import CodeAgent, HfApiModel
from smolagents.tools import Tool
import os

# Definir la clase Fibonacci heredando de Tool
class FibonacciTool(Tool):
    name = "fibonacci"  # Agregamos el atributo name
    description = "Calculates the nth Fibonacci number."
    inputs = {
        "n": {
            "type": "integer",
            "description": "The position in the Fibonacci sequence to calculate"
        }
    }
    output_type = "integer"  # Añadimos el tipo de salida
     
    def forward(self, n: int) -> int:
        """Calcula el enésimo número de Fibonacci."""
        a, b = 0, 1
        for _ in range(n - 1):
            a, b = b, a + b
        return a

# Crear una instancia de la herramienta
fibonacci_tool = FibonacciTool()


# Usar HfApiModel con una clave de Hugging Face
# hf_key = os.getenv("HF_TOKEN")  # Asegúrate de definir esta variable en tu entorno
# model_id = "meta-llama/Llama-3.3-70B-Instruct"  # Puedes cambiarlo por otro modelo disponible

model = HfApiModel(model_id=model_id, token=hf_key)

# Configurar el agente con la tool de Fibonacci
agent = CodeAgent(tools=[fibonacci_tool], model=model, add_base_tools=True)

# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 118th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = fibonacci(n=118)                                                                                        
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
1264937032042997393488322

Out: None

[Step 0: Duration 0.18 seconds| Input tokens: 2,166 | Output tokens: 46]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(1264937032042997393488322)                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1264937032042997393488322

[Step 1: Duration 0.31 seconds| Input tokens: 4,464 | Output tokens: 105]

Response: 1264937032042997393488322


### Inspecting an agent run

Here are a few useful attributes to inspect what happened after a run:

agent.logs
Stores the fine-grained logs of the agent. At every step of the agent’s run, everything gets stored in a dictionary that then is appended to agent.logs.

Running agent.write_memory_to_messages() writes the agent’s memory as list of chat messages for the Model to view. This method goes over each step of the log and only stores what it’s interested in as a message: for instance, it will save the system prompt and task in separate messages, then for each step it will store the LLM output as a message, and the tool call output as another message. Use this if you want a higher-level view of what has happened - but not every log will be transcripted by this method.

In [45]:
agent.logs

The 'logs' attribute is deprecated and will soon be removed. Please use 'self.memory.steps' instead.


[SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using 

### Utilidades

### 1. Buscar en internet conDuckDuckGo

In [80]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
print(search_tool("Who's the current president of Russia?"))

## Search Results

[List of presidents of Russia - Wikipedia](https://en.wikipedia.org/wiki/List_of_presidents_of_Russia)
The office of the president of Russia is the highest authority in the Russian Federation.The holder is the federation's head of state and has formal presidency over the State Council as well as being the commander in chief of the Russian Armed Forces.The office was introduced in 1918 after the February Revolution with the current office emerging after a referendum of 1991. [1]

[President of Russia - Wikipedia](https://en.wikipedia.org/wiki/President_of_Russia)
The president of Russia, officially the president of the Russian Federation (Russian: Президент Российской Федерации, romanized: Prezident Rossiyskoy Federatsii), is the executive head of state of Russia.The president is the chair of the Federal State Council and the supreme commander-in-chief of the Russian Armed Forces.It is the highest office in Russia.

[Vladimir Putin - Wikipedia](https://en.wikipedia.or